## Data Science Discovering Structure and Opportunity via Network Modeling 
![](images/twittercloud.jpg)

In [14]:
#!pip uninstall visJS2Jupyter -y

In [15]:
#!pip install git+https://github.com/albre116/visJS2jupyter

In [16]:
###lets take a peek 
import requests
import html
import subprocess
from IPython.display import display, HTML, IFrame
html_template = html.escape(requests.get('https://twitter.com/GS1_US/status/1043149189864144897').text)
display(HTML("<iframe srcdoc='{}' height='600' width='1000'>".format(html_template)))

In [17]:
Actor1 = "GS1Foodservice"
html_template = html.escape(requests.get('https://twitter.com/{}'.format(Actor1)).text)
display(HTML("<iframe srcdoc='{}' height='600' width='1000'>".format(html_template)))

# More about GS1 standards
- https://www.gs1us.org/industries/retail-grocery/standards-in-use/food-safety?utm_source=twitter&utm_medium=social&utm_term=NationalFoodSafetyMonth&utm_content=NationalFoodSafetyMonth&utm_campaign=FoodSafetyPage

## What the profile page looks like to a computer using API (Application Programming Interface)

In [18]:

###get profile page
import sys
import os
import importlib
sys.path.insert(0,"twitterpackage/Chap02_03")
#print(sys.path)
#print(os.environ)

from twitter_client import get_twitter_client
import json
client = get_twitter_client()
profile = client.get_user(screen_name=Actor1)
print(json.dumps(profile._json, indent =4))

{
    "id": 601375154,
    "id_str": "601375154",
    "name": "GS1 US Foodservice",
    "screen_name": "GS1Foodservice",
    "location": "Lawrenceville, New Jersey",
    "profile_location": null,
    "description": "The official twitter page for the @GS1_US #Foodservice #Standards Initiative. #GS1Standards",
    "url": "http://t.co/kudtXwWh0o",
    "entities": {
        "url": {
            "urls": [
                {
                    "url": "http://t.co/kudtXwWh0o",
                    "expanded_url": "http://www.gs1us.org/foodservice",
                    "display_url": "gs1us.org/foodservice",
                    "indices": [
                        0,
                        22
                    ]
                }
            ]
        },
        "description": {
            "urls": []
        }
    },
    "protected": false,
    "followers_count": 1015,
    "friends_count": 577,
    "listed_count": 49,
    "created_at": "Wed Jun 06 23:26:00 +0000 2012",
    "favourites_count

## Lets look at some basic questions
* ### What topics are being Tweeted on the Front page this year?
* ### How Wide is the Social Media Reach?
* ### Who are the influencers in the social media network?


In [19]:
#cmd = "python twitterpackage/Chap02_03/twitter_get_user.py {}".format(Actor1)
#print(subprocess.check_output(cmd, shell=True).decode())

#cmd = "python ./twitterpackage/Chap02_03/twitter_get_user_timeline_daterange.py --username {} --startdate {}".format(Actor1,"2018-01-01")
#print(subprocess.check_output(cmd, shell=True).decode())

In [20]:
print("what hashtag topics are on {}'s mind?".format(Actor1))
cmd = "python twitterpackage/Chap02_03/twitter_hashtag_frequency.py user_timeline_{}.jsonl".format(Actor1)
print(subprocess.check_output(cmd, shell=True).decode())


what hashtag topics are on GS1Foodservice's mind?
foodservice: 86
blockchain: 80
gs1standards: 78
data: 77
traceability: 75
supplychain: 72
barcode: 42
barcodes: 40
quiz: 26
foodsafety: 25
technology: 24
dataquality: 24
gs1connect18: 20
digital: 18
transparency: 17
dcforum18: 12
gtin: 10
smallbusiness: 9
iot: 9
mealkit: 8
ecommerce: 7
grocery: 6
smallbusinesses: 5
baseball: 5
fbf: 5
genz: 5
barcoding: 5
mobile: 5
mealkits: 4
cleanlabel: 4
gs1connect19: 4
recall: 4
tbt: 4
millennials: 4
omnichannel: 3
sustainability: 3
tech: 3
retailers: 3
rfid: 3
jobs: 2



In [21]:
print("What do the follower statistics look like for {}?".format(Actor1))
cmd = "python twitterpackage/Chap02_03/twitter_followers_stats.py {}".format(Actor1)
print(subprocess.check_output(cmd, shell=True).decode())

What do the follower statistics look like for GS1Foodservice?
----- Timing -----
Initialize data: 0.08478713035583496
Set-based operations: 0.03240323066711426
Total time: 0.11719036102294922
----- Stats -----
GS1Foodservice has 1015 followers
GS1Foodservice has 577 friends
GS1Foodservice has 124 mutual friends
453 friends are not following GS1Foodservice back
891 followers are not followed back by GS1Foodservice
Some mutual friends: FoodSafetyMag, ConserveNow, FMI_ORG, gs1ineurope, TheBarCodeNews



In [22]:
###lets get all mutual friends (only run once)
# print("Lets pull the mutual friends for {}".format(Actor1))
# cmd = "python twitterpackage/Chap02_03/twitter_mutual_friend_followers.py {}".format(Actor1)
# print(subprocess.check_output(cmd, shell=True).decode())

In [42]:
####Generate Mutual Friend Network
####These are the interconnections
import os
import sys
import json
import time
import math
from tweepy import Cursor
from twitter_client import get_twitter_client
import sys
import json
from random import sample
import sys
import json
from operator import itemgetter
import networkx as nx
import matplotlib.pyplot as plt
import urllib
import requests
import html
import numpy as np

Refresh = True
Target = Actor1
if Refresh == True:
    followers_file = 'users/{}/followers.jsonl'.format(Target)
    friends_file = 'users/{}/friends.jsonl'.format(Target)
    with open(followers_file) as f1, open(friends_file) as f2:
        t0 = time.time()
        followers = []
        friends = []
        for line in f1:
            profile = json.loads(line)
            followers.append(profile['screen_name'])
        for line in f2:
            profile = json.loads(line)
            friends.append(profile['screen_name'])
        t1 = time.time()
        mutual_friends = [user for user in friends if user in followers]
        followers_not_following = [user for user in followers if user not in friends]
        friends_not_following = [user for user in friends if user not in followers]
        t2 = time.time()
        print("Building Mutual Friend Network for {} Who Has {} Mutual Friends".format(Target, len(mutual_friends)))

    #mutual_friends = mutual_friends[:5]
    G = nx.DiGraph()
    for mutual in mutual_friends:
        #G.add_edge(screen_name,mutual)
        #G.add_edge(mutual,screen_name)
        G.add_node(mutual)

    ###Link the mutual friends network
    for mutual in mutual_friends:
        screen_name = mutual
        followers_file = 'users/{}/followers.jsonl'.format(screen_name)
        friends_file = 'users/{}/friends.jsonl'.format(screen_name)
        try:
            with open(followers_file) as f1, open(friends_file) as f2:
                followers = []
                friends = []
                for line in f1:
                    profile = json.loads(line)
                    followers.append(profile['screen_name'])
                ###Add to the NetworkX Graph those that are connected
                followers_in_network = list(set(followers).intersection(mutual_friends))
                for u in followers_in_network:
                    G.add_edge(u,screen_name)
                print("{} followers in network for {}".format(len(followers_in_network),screen_name))

                for line in f2:
                    profile = json.loads(line)
                    friends.append(profile['screen_name'])
                ###Add to the NetworkX Graph those that are connected
                friends_in_network = list(set(friends).intersection(mutual_friends))
                for u in friends_in_network:
                    G.add_edge(screen_name,u)
                print("{} friends in network for {}".format(len(friends_in_network),screen_name))
                
                ### We can cut this down to just those whom are mutual for this analysis
                #mutual_friends_subordinate = [user for user in friends if user in followers]
                #mutual_friends_in_network = list(set(mutual_friends_subordinate).intersection(mutual_friends))
                #for u in mutual_friends_in_network:
                #    G.add_edge(screen_name,u)
                #print("{} mutual friends in network for {}".format(len(mutual_friends_in_network),screen_name))
                
        except OSError:
            print("no file there for {}, removing node".format(screen_name))
            G.remove_node(screen_name)
    nx.write_gpickle(G,"networkX{}.gpickle".format(Target))
else:
    G = nx.read_gpickle("networkX{}.gpickle".format(Target))


Building Mutual Friend Network for GS1Foodservice Who Has 124 Mutual Friends
0 followers in network for GetBrandedcom
0 friends in network for GetBrandedcom
11 followers in network for BungeMOE
25 friends in network for BungeMOE
22 followers in network for Reinhart_Corp
28 friends in network for Reinhart_Corp
6 followers in network for MerchantsFS
8 friends in network for MerchantsFS
20 followers in network for pma
16 friends in network for pma
17 followers in network for PRKaitlin
41 friends in network for PRKaitlin
3 followers in network for McCainFoodsJOBS
6 friends in network for McCainFoodsJOBS
16 followers in network for LoftwareInc
24 friends in network for LoftwareInc
0 followers in network for edg1972
0 friends in network for edg1972
5 followers in network for Ari_Greenberg
5 friends in network for Ari_Greenberg
0 followers in network for BCurran_FS
10 friends in network for BCurran_FS
9 followers in network for juliemcgill67
37 friends in network for juliemcgill67
10 follower

30 followers in network for gs1ksa
15 friends in network for gs1ksa
31 followers in network for GS1_Latvija
34 friends in network for GS1_Latvija
30 followers in network for gs1pe
28 friends in network for gs1pe
38 followers in network for gs1ineurope
25 friends in network for gs1ineurope
33 followers in network for GS1nederland
18 friends in network for GS1nederland
34 followers in network for GS1argentina
35 friends in network for GS1argentina
48 followers in network for gs1uk
37 friends in network for gs1uk
33 followers in network for GS1Germany
27 friends in network for GS1Germany
7 followers in network for WeRRestaurants
34 friends in network for WeRRestaurants
26 followers in network for compassgroupusa
18 friends in network for compassgroupusa
15 followers in network for gs1usAnnette
10 friends in network for gs1usAnnette
2 followers in network for miked51
8 friends in network for miked51
39 followers in network for IFMAWORLD
43 friends in network for IFMAWORLD
45 followers in n

# Lets first look at the "hairball of connections"

In [43]:
import matplotlib as mpl
import visJS2jupyter.visJS_module
displaylabel = {n:'@' + n for n in G.nodes()}
#Create network layout for visualizations
pos = nx.spring_layout(G , scale= 1)
#plt.axis("off")
#nx.draw_networkx(G, pos = pos, with_labels = False, arrows=False, node_size = 35)


Gplot = G.copy()

pctplot = 1
for k,v in G.degree:
    tmp = G.edges(nbunch=[k])
    ns = int(len(tmp)*(1-pctplot))
    if ns > 0:
        yank = np.random.choice(range(len(tmp)),ns,replace=False)
    else:
        yank = list()
    tmpp = list(tmp)
    Gplot.remove_edges_from([tmpp[i] for i in yank])


nodes = Gplot.nodes()
edges = list(Gplot.edges())
nodes_dict = [{"id":n,
               "color":'black',
              "title": displaylabel[n],
               "displaylabel":n,
              "x":pos[n][0]*1000,
              "y":pos[n][1]*1000} for n in nodes
              ]
node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
              "color":"gray"
              } for i in range(len(edges))]

# set some network-wide styles
NodeGraph = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                          node_label_field='displaylabel',
                          node_size_multiplier=10,
                          node_size_transform = '',
                          node_color_highlight_border='red',
                          node_color_highlight_background='#D3918B',
                          node_color_hover_border='blue',
                          node_color_hover_background='#8BADD3',
                          node_font_size=25,
                          edge_arrow_to=False,
                          physics_enabled=True,
                          edge_color_highlight='#8A324E',
                          edge_color_hover='#8BADD3',
                          edge_label_field='label',
                          edge_title_field='title',
                          edge_width=3,
                          max_velocity=15,
                          min_velocity=1,
                          graph_id = 1)

NodeGraph

# Lets find the Top X influencers in the Graph
- betweenness centrality identifies those that have the most direct connections passing through them between actors

In [44]:
###lets find the most central 5 members in the network
###using betweenness centrality
bt = nx.betweenness_centrality(G)
nx.set_node_attributes(G, name='betweenness', values=bt)
nx.set_node_attributes(G, name='size', values=5)
nx.set_node_attributes(G, name='color', values="black")

###number of influencers to identify
top = 3
max_nodes =  sorted(bt.items(), key = lambda v: -v[1])[:top]

influencerlabel = displaylabel

for max_key, max_val in max_nodes:
    G.node[max_key]['size'] = 15
    G.node[max_key]['color'] = 'yellow'
    html_template = html.escape(requests.get('https://twitter.com/{}'.format(max_key)).text)
    influencerlabel[max_key] = "<iframe srcdoc='{}' height='600' width='800'>".format(html_template)
    

color = nx.get_node_attributes(G,'color')
size = nx.get_node_attributes(G,'size')

Gplot = G.copy()

pctplot = 1
for k,v in G.degree:
    tmp = G.edges(nbunch=[k])
    ns = int(len(tmp)*(1-pctplot))
    if ns > 0:
        yank = np.random.choice(range(len(tmp)),ns,replace=False)
    else:
        yank = list()
    tmpp = list(tmp)
    Gplot.remove_edges_from([tmpp[i] for i in yank])

nodes = Gplot.nodes()
edges = list(Gplot.edges())
nodes_dict = [{"id":n,
               "color":color[n],
              "title": influencerlabel[n],
              #"displaylabel":n,
              "degree":size[n],
              "x":pos[n][0]*1000,
              "y":pos[n][1]*1000} for n in nodes
              ]
node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
              "color":"gray"
              } for i in range(len(edges))]

# set some network-wide styles
NodeGraphInfluencer = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                          tooltip_delay=300,
                          node_label_field='displaylabel',
                          node_size_multiplier=10,
                          node_color_highlight_border='red',
                          node_color_highlight_background='#D3918B',
                          node_color_hover_border='blue',
                          node_color_hover_background='#8BADD3',
                          node_font_size=25,
                          edge_arrow_to=False,
                          physics_enabled=True,
                          edge_color_highlight='#8A324E',
                          edge_color_hover='#8BADD3',
                          edge_label_field='label',
                          edge_title_field='title',
                          edge_width=3,
                          max_velocity=15,
                          min_velocity=1,
                          graph_id = 2)

NodeGraphInfluencer

# Lets Identify Community Structure of Interconnected Networks
- Compares within community connection density to outside community connection density

In [48]:
import community

Gc = G.to_undirected()
parts = community.best_partition(Gc)
ranges = list(set([parts.get(node) for node in Gc.nodes()]))

nx.set_node_attributes(Gc, name='community', values=parts)
nx.set_node_attributes(Gc, name='size', values=5)

influencerlabel = displaylabel
for c in ranges:
    bunches = [k for k,v in nx.get_node_attributes(Gc,name='community').items() if v == c]
    if len(bunches) > 1:
        bt = nx.betweenness_centrality(G.subgraph(bunches))
        ###number of influencers to identify
        top = 1
        max_nodes =  sorted(bt.items(), key = lambda v: -v[1])[:top]
        for max_key, max_val in max_nodes:
            Gc.node[max_key]['size'] = 15
            Gc.node[max_key]['color'] = 'yellow'
            html_template = html.escape(requests.get('https://twitter.com/{}'.format(max_key)).text)
            influencerlabel[max_key] = "<iframe srcdoc='{}' height='600' width='800'>".format(html_template)

color = visJS2jupyter.visJS_module.return_node_to_color(Gc,field_to_map='community',cmap=plt.get_cmap("Spectral"))

size = nx.get_node_attributes(Gc,'size')

Gplot = Gc.copy()

pctplot = 1
for k,v in G.degree:
    tmp = G.edges(nbunch=[k])
    ns = int(len(tmp)*(1-pctplot))
    if ns > 0:
        yank = np.random.choice(range(len(tmp)),ns,replace=False)
    else:
        yank = list()
    tmpp = list(tmp)
    Gplot.remove_edges_from([tmpp[i] for i in yank])

nodes = Gplot.nodes()
edges = list(Gplot.edges())

nodes_dict = [{"id":n,
               "color":color[n],
              "title": influencerlabel[n],
              #"displaylabel":n,
              "degree":size[n],
              "x":pos[n][0]*1000,
              "y":pos[n][1]*1000} for n in nodes
              ]
node_map = dict(zip(nodes,range(len(nodes))))  # map to indices for source/target in edges

edges_dict = [{"source":node_map[edges[i][0]], 
               "target":node_map[edges[i][1]], 
              "color":"gray"
              } for i in range(len(edges))]

# set some network-wide styles
NodeGraphInfluencer = visJS2jupyter.visJS_module.visjs_network(nodes_dict,edges_dict,
                           tooltip_delay=300,
                          node_label_field='displaylabel',
                          node_size_multiplier=10,
                          node_color_highlight_border='red',
                          node_color_highlight_background='#D3918B',
                          node_color_hover_border='blue',
                          node_color_hover_background='#8BADD3',
                          node_font_size=25,
                          edge_arrow_to=False,
                          physics_enabled=True,
                          edge_color_highlight='#8A324E',
                          edge_color_hover='#8BADD3',
                          edge_label_field='label',
                          edge_title_field='title',
                          edge_width=1,
                          max_velocity=15,
                          min_velocity=1,
                          graph_id = 3)

NodeGraphInfluencer

Error: edges_dict is empty
